In [69]:
# Basic setup and sanity checking
%run ./00_setup.ipynb

Python version: 3.6.6
Pandas version: 0.23.4
Numpy version: 1.15.1


## Action Authoring and Deployment
A Cortex Action can be authored and deployed from within a notebook for rapid prototyping and testing.  The Cortex SDK for Python includes an iPython Magic called `%%cortex_action` that takes a notebook cell and deploys it as a Cortex Action.

In [78]:
%%cortex_action --name c12e/image-classify --function classify --daemon --requirements watson_developer_cloud

from cortex import Message
from watson_developer_cloud import VisualRecognitionV3

def classify(params):

    msg = Message(params)

    image_url = msg.payload.get('imageUrl')
    apiKey = msg.payload.get('apiKey')
    classifierId = msg.payload.get('classifierId')

    if classifierId is None or classifierId == "":
        return {"payload": "Classifier ID Not Provided"}

    visual_recognition = VisualRecognitionV3('2018-03-19', iam_apikey=apiKey)

    response = visual_recognition.classify(url=image_url, classifier_ids=[classifierId], threshold="0.5").get_result()

    if response is not None:
        return {"payload": response}
    else:
        return {"payload": "Unable to get response from Watson Visual Recognition"}

Building Cortex Action (daemon): c12e/image-classify
Building Docker image private-registry.cortex.insights.ai/lkrishna-prod/c12e_image-classify:wi0h7g0...
Step 1/13 : FROM continuumio/miniconda3:4.5.4
Step 2/13 : WORKDIR /app
Step 3/13 : RUN apt-get update && apt-get install -y linux-headers-amd64 build-essential
Step 4/13 : RUN conda config --add channels conda-forge
Step 5/13 : RUN conda install --yes flask gunicorn
Step 6/13 : COPY conda_requirements.txt .
Step 7/13 : RUN conda install --yes --file conda_requirements.txt
Step 8/13 : RUN pip install "dill==0.2.8.2" "cortex-client==5.4.6"
Step 9/13 : COPY requirements.txt .
Step 10/13 : RUN pip install -r requirements.txt
Step 11/13 : COPY action.py .
Step 12/13 : EXPOSE 5000
Removing intermediate container d831775f0156
Step 13/13 : ENTRYPOINT ["gunicorn", "-b", "0.0.0.0:5000", "-t", "60", "action:app"]
Removing intermediate container ab8152da5882
Successfully built 1e16ad9318fc
Successfully tagged private-registry.cortex.insights.ai

Name,Version,Type,Kind,Image,Deployment Status,c12e/image-classify,1,daemon,None,private-registry.cortex.insights.ai/lkrishna-prod/c12e_image-classify:wi0h7g0,STARTED


## Local Testing
Once the Action is built, we can run both a local and remote test.  The local test in our example can be done by simply calling the `detect` function we defined above.

In [95]:
import sys
import argparse
from pprint import pprint


def test_detect(args):
    imageUrl = 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg'
    classifierId = 'Connectors_227384960'
    apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
    m = Message.with_payload({'imageUrl': imageUrl,'apiKey':apiKey,'classifierId':classifierId})
    result = classify(m.to_params())
    pprint(result['payload'])

test_detect(None)

{'custom_classes': 4,
 'images': [{'classifiers': [{'classes': [{'class': 'usb_male',
                                           'score': 0.882}],
                              'classifier_id': 'Connectors_227384960',
                              'name': 'Connectors'}],
             'resolved_url': 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg',
             'source_url': 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg'}],
 'images_processed': 1}


## Remote Testing
Using the Cortex client, we can execute a remote test of our Action to make sure it deployed as expected.

In [101]:
client = Cortex.client()
action = client.action('c12e/image-classify')
action

Name,Version,Type,Kind,Image,Deployment Status,c12e/image-classify,1,daemon,None,private-registry.cortex.insights.ai/lkrishna-prod/c12e_image-classify:wi0h7g0,COMPLETED (running)


In [174]:
imageUrl = 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg'
classifierId = 'Connectors_227384960'
apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
params = {'payload': {'imageUrl': imageUrl,'apiKey':apiKey,'classifierId':classifierId}}
rs = action.invoke(Message.with_payload(params.get('payload')))
result = rs.payload
classifiers =  result.get("images")
print (classifiers[0].get('classifiers'))

[{'classifier_id': 'Connectors_227384960', 'name': 'Connectors', 'classes': [{'class': 'usb_male', 'score': 0.882}]}]
